# Overview of embeddings-based retrieval

In this lesson we'll index a PDF. We'll have the vector database calculate the embeddings and pass the information to the llm as additional information.

## Installation

In [25]:
%pip install -q -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Reading the text from a PDF

You can view the pdf in your browser [here](./microsoft_annual_report_2022.pdf) if you would like. 

In [26]:
from pypdf import PdfReader

reader = PdfReader("data/microsoft_annual_report_2022.pdf")

# Reading the pages and extracting the text
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filte out the empty text strings
pdf_texts = [text for text in pdf_texts if text]

#print(word_wrap(pdf_texts[0]))
# Print out the first piece
print(pdf_texts[0])

1 Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and responsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 
company is better positioned to help th

**Note: Take a look at the first page of the document**
(pdf_texts[0] is the first page)

# Load Langchain text splitter tools

### Recursive Character splitter
Splitting text by recursively look at characters.

Recursively tries to split by different characters to find one that works.


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

*Note: Because the way we read in the pdf via text there are no paragraphs so chunking doesn't really work* 

This splitter works more mechanically by recursively splitting text based on a set of delimiters, typically starting with larger units (e.g., paragraphs), then moving to smaller units (e.g., sentences, words) until the desired chunk size is met.

In [28]:
character_splitter = RecursiveCharacterTextSplitter(
    #separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0,
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

# print the first 5 chunks
for i in range(5):
    print("Chunk#=" + str(i) + " Chunk=" + character_split_texts[i],"\n")

print(f"\nTotal chunks: {len(character_split_texts)}")

Chunk#=0 Chunk=1 Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and responsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 

Chunk#=1 Chunk=company 


For Example: chunk0

- 960 characters
- 155 words
- 193 tokens

[Check out the OpenAI tokenizer](https://platform.openai.com/tokenizer)



## SentenceTransformersTokenTextSplitter
Splitting text to tokens using sentence model tokenizer. The default model is `sentence-transformers/all-mpnet-base-v2'`

This splitter is designed to break down text using semantic units, such as sentences or clauses, often based on token counts (where a token could be a word, subword, or punctuation mark). It uses models from the SentenceTransformers library to understand and split text in a semantically meaningful way.

In [29]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

# print the first 5 chunks
for i in range(5):
    print("Chunk#=" + str(i) + " Chunk=",token_split_texts[i],"\n")
print(f"\nTotal chunks: {len(token_split_texts)}")

Chunk#=0 Chunk= 1 dear shareholders, colleagues, customers, and partners : we are living through a period of historic economic, societal, and geopolitical change. the world in 2022 looks nothing like the world in 2019. as i write this, inflation is at a 40 - year high, supply chains are stretched, and the war in ukraine is ongoing. at the same time, we are entering a technological era with the potential to power awesome advancements across every sector of our economy and society. as the world ’ s largest software company, this places us at a historic intersection of opportunity and responsibility to the world around us. our mission to empower every person and every organization on the planet to achieve more has never been more urgent or more necessary. for all the uncertainty in the world, one thing is clear : people and organizations in every industry are increasingly looking to digital technology to overcome today ’ s challenges and emerge stronger. and no 

Chunk#=1 Chunk= company i

Note: for cleanup rm -rf ~/.cache/huggingface/hub/models--sentence-transformers--a

## Setting up the embeddings

In [30]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
model = embedding_function.models
print("Model used for embeddings:")
print(model)

Model used for embeddings:
{'all-MiniLM-L6-v2': SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)}


## Calculate embedding of the first chunk

In [31]:
embedding = embedding_function([token_split_texts[0]])
print(embedding)

[[-0.05130084231495857, -0.011773279868066311, 0.04940269514918327, -0.07973462343215942, 0.01482198853045702, -0.017085712403059006, -0.03712102025747299, 0.010243200697004795, 0.029443394392728806, -0.01155102625489235, -0.029306042939424515, 0.059875961393117905, 0.04193371161818504, 0.0009110003593377769, -0.00024818183737806976, 0.055705610662698746, -0.095044806599617, -0.1123923659324646, -0.04467058181762695, -0.0028009042143821716, -0.024321122094988823, -0.01566249318420887, -0.049685437232255936, 0.017255669459700584, -0.041929569095373154, 0.02931785397231579, -0.021453693509101868, -0.07143814861774445, -0.03475289046764374, 0.021638259291648865, -0.015290451236069202, 0.05068206414580345, 0.029244009405374527, 0.04425172507762909, 0.03545272722840309, 0.022688942030072212, 0.06838447600603104, -0.020330004394054413, 0.01909978687763214, -0.0992356464266777, -0.011153746396303177, -0.12798883020877838, -0.03553144633769989, 0.016359351575374603, 0.06487606465816498, -0.018

## Setting up the ChromaDB vector database

In [32]:
#chroma_client = chromadb.Client()
#chroma_client = chromadb.Client(persist_directory="../data/chroma_db/")
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

chroma_client = chromadb.PersistentClient(
    path="data/chroma_db/",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

COLLECTION_NAME = "microsoft_annual_report_2022"

# Removing any existing collection
#collection = chroma_client.get_collection(COLLECTION_NAME)
#if (collection):
#    chroma_client.delete_collection(COLLECTION_NAME)

# The default is Euclidean distance
#chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function)
chroma_collection = chroma_client.create_collection(COLLECTION_NAME, embedding_function=embedding_function,metadata={"hnsw:space": "cosine"})

## Index all chunks in the vector database

In [33]:
ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)

# Print the number of documents indexed
chroma_collection.count()

349

## Helper function to print the results our vector search

In [34]:
def print_results_and_documents(results, retrieved_documents, word_wrap):
    """
    Prints keys and values from the results dictionary and documents with word wrapping.

    Args:
        results (dict): A dictionary where keys are strings and values are either strings or lists.
        retrieved_documents (list): A list of documents to be printed.
        word_wrap (function): A function to apply word wrapping to the documents.

    Returns:
        None
    """
    # Iterate through the dictionary and print each key with its associated value
    for key, value in results.items():
        print(f"{key}:")

        # Check if the value is a list and print its elements
        if isinstance(value, list):
            for i, item in enumerate(value):
                print(f"  Item {i+1}: {item}")
        else:
            # Directly print the value if it's not a list
            print(f"  {value}")

        print()  # Add a newline for better readability

    # Iterate through the list of documents and print each one with word wrapping
    for document in retrieved_documents:
        print(word_wrap(document))
        print('\n')

## Searching the vector database

In [35]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=query,
                                   n_results=5, 
                                   include=['documents', 'embeddings', "distances"])

retrieved_documents = results['documents'][0]

from helper_utils import word_wrap
print_results_and_documents(results, retrieved_documents, word_wrap)

ids:
  Item 1: ['321', '293', '331', '319', '194']

distances:
  Item 1: [0.3957172876850713, 0.4243314862251282, 0.4562008321326718, 0.4578593225401921, 0.4580157399177551]

metadatas:
  None

embeddings:
  Item 1: [[0.006851615849882364, -0.07879453152418137, -0.013295024633407593, -0.018663959577679634, -0.01320614479482174, 0.05207332223653793, 0.024349980056285858, 0.029772840440273285, 0.0398593433201313, 0.08490502089262009, -0.0272703617811203, 0.04178402200341225, 0.050083816051483154, -0.021252332255244255, -0.029229924082756042, 0.003365496639162302, 0.025213973596692085, -0.11350663006305695, 0.02950187586247921, -0.01701783761382103, -0.014468980953097343, 0.03350761905312538, -0.055656228214502335, -0.038956690579652786, 0.02398926019668579, 0.006582111585885286, -0.054270368069410324, -0.02146941050887108, -0.015810992568731308, -0.06164819747209549, -0.07168304920196533, 0.031110158190131187, -0.007594883441925049, 0.08013573288917542, 0.0024636457674205303, -0.07940092

**Questions for understanding:**
- what are the chromadb query output fields
- how does a chromadb query find the documents
- Can you explain how the similarity search works


## Using the RAG documents in the LLM prompt

### Setting up the LLM


In [36]:
from openai import OpenAI
openai_client = OpenAI()

## Mixing the retrieved documents in the prompt

In [37]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [38]:
output = rag(query=query, retrieved_documents=retrieved_documents)

In [39]:
print(word_wrap(output))

The total revenue for the year ended June 30, 2022, was $198,270
million.
